In [ ]:
import torch
import torchvision

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
device

'cuda'

In [ ]:
import torchvision
from torchvision import transforms

# Transformaciones comunes para preprocesamiento
transform = transforms.Compose([
    transforms.Resize((64, 64)),      # Redimensionar a 64x64 píxeles
    transforms.ToTensor(),            # Convertir a tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizar
])

In [ ]:
# Crear dataloaders para el conjunto de entrenamiento y prueba
dataloader = {
    'train': torch.utils.data.DataLoader(
        torchvision.datasets.SVHN(root='./data', split='train', download=True, transform=transform),
        batch_size=64, shuffle=True, pin_memory=True),
    'test': torch.utils.data.DataLoader(
        torchvision.datasets.SVHN(root='./data', split='test', download=True, transform=transform),
        batch_size=64, shuffle=False, pin_memory=True)
}

Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


In [ ]:
def block(c_in, c_out, k=5, p=2, s=2, pk=2, ps=1):
    return torch.nn.Sequential(
        torch.nn.Conv2d(c_in, c_out, k, padding=p, stride=s),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(pk, stride=ps)
    )



class CNN(torch.nn.Module):
  def __init__(self, n_channels=3, n_outputs=10):
    super().__init__()
    self.conv1 = block(n_channels, 64)
    self.conv2 = block(64, 128)
    self.conv3 = block(128, 256)
    self.conv4 = block(256, 512)
    self.fc = torch.nn.Linear(512 * 3 * 3, n_outputs)


  def forward(self, x):
    print("Dimensiones:")
    print("Entrada: ", x.shape)
    x = self.conv1(x)
    print("conv1: ", x.shape)
    x = self.conv2(x)
    print("conv2: ", x.shape)
    x = self.conv3(x)
    print("conv3: ", x.shape)
    x = self.conv4(x)
    print("conv4: ", x.shape)
    x = x.view(x.shape[0], -1)
    print("pre fc: ", x.shape)
    x = self.fc(x)
    print("Salida: ", x.shape)
    return x

In [ ]:
model = CNN()

output = model(torch.randn(64, 3, 64, 64))

Dimensiones:
Entrada:  torch.Size([64, 3, 64, 64])
conv1:  torch.Size([64, 64, 31, 31])
conv2:  torch.Size([64, 128, 15, 15])
conv3:  torch.Size([64, 256, 7, 7])
conv4:  torch.Size([64, 512, 3, 3])
pre fc:  torch.Size([64, 4608])
Salida:  torch.Size([64, 10])


In [ ]:
class CNN(torch.nn.Module):
  def __init__(self, n_channels=3, n_outputs=10):
    super().__init__()
    self.conv1 = block(n_channels, 64)
    self.conv2 = block(64, 128)
    self.conv3 = block(128, 256)
    self.conv4 = block(256, 512)
    self.fc = torch.nn.Linear(512*3*3, n_outputs)

  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.conv4(x)
    x = x.view(x.shape[0], -1)
    x = self.fc(x)
    return x

In [ ]:
from tqdm import tqdm
import numpy as np

def fit(model, dataloader, epochs=5):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):
        model.train()
        train_loss, train_acc = [], []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
            train_acc.append(acc)
            bar.set_description(f"loss {np.mean(train_loss):.5f} acc {np.mean(train_acc):.5f}")
        bar = tqdm(dataloader['test'])
        val_loss, val_acc = [], []
        model.eval()
        with torch.no_grad():
            for batch in bar:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
                val_acc.append(acc)
                bar.set_description(f"val_loss {np.mean(val_loss):.5f} val_acc {np.mean(val_acc):.5f}")
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f} acc {np.mean(train_acc):.5f} val_acc {np.mean(val_acc):.5f}")

In [ ]:
model = CNN()
fit(model, dataloader)

val_loss 0.67815 val_acc 0.78902: 100%|██████████| 407/407 [00:18<00:00, 22.44it/s]


Epoch 1/5 loss 1.42242 val_loss 0.67815 acc 0.50830 val_acc 0.78902


val_loss 0.50782 val_acc 0.84379: 100%|██████████| 407/407 [00:17<00:00, 23.22it/s]


Epoch 2/5 loss 0.55648 val_loss 0.50782 acc 0.82842 val_acc 0.84379


val_loss 0.47148 val_acc 0.85931: 100%|██████████| 407/407 [00:17<00:00, 22.92it/s]


Epoch 3/5 loss 0.46267 val_loss 0.47148 acc 0.85872 val_acc 0.85931


val_loss 0.45978 val_acc 0.86338: 100%|██████████| 407/407 [00:17<00:00, 22.90it/s]


Epoch 4/5 loss 0.41025 val_loss 0.45978 acc 0.87301 val_acc 0.86338


val_loss 0.45663 val_acc 0.86457: 100%|██████████| 407/407 [00:17<00:00, 22.84it/s]

Epoch 5/5 loss 0.37696 val_loss 0.45663 acc 0.88343 val_acc 0.86457
